In [1]:
import ctypes
import json
import requests
import csmapi
import os

In [2]:
# Set up the location of the usgscsm library:
lib = "$CONDA_PREFIX/lib/libusgscsm.so"
ctypes.CDLL(os.path.expandvars(lib))

<CDLL '/home/kberry/anaconda3/envs/csm/lib/libusgscsm.so', handle 5622855ab1b0 at 0x7f9391026650>

In [3]:
label = """
PDS_VERSION_ID                     = PDS3

/*FILE CHARACTERISTICS*/
RECORD_TYPE                        = FIXED_LENGTH
RECORD_BYTES                       = 5064      
FILE_RECORDS                       = 52225     
LABEL_RECORDS                      = 1         
^IMAGE                             = 2         

/*DATA IDENTIFICATION*/
DATA_SET_ID                        = "LRO-L-LROC-2-EDR-V1.0"
ORIGINAL_PRODUCT_ID                = nacr0002fc2d
PRODUCT_ID                         = M128962330RE
MISSION_NAME                       = "LUNAR RECONNAISSANCE ORBITER"
MISSION_PHASE_NAME                 = "NOMINAL MISSION"
INSTRUMENT_HOST_NAME               = "LUNAR RECONNAISSANCE ORBITER"
INSTRUMENT_HOST_ID                 = LRO
INSTRUMENT_NAME                    = "LUNAR RECONNAISSANCE ORBITER CAMERA"
INSTRUMENT_ID                      = LROC
LRO:PREROLL_TIME                   = 2010-05-20T02:37:44.123
START_TIME                         = 2010-05-20T02:37:44.470
STOP_TIME                          = 2010-05-20T02:38:02.180
LRO:SPACECRAFT_CLOCK_PREROLL_COUNT = "1/296015864:06553"
SPACECRAFT_CLOCK_START_COUNT       = "1/296015864:29310"
SPACECRAFT_CLOCK_STOP_COUNT        = "1/296015882:10315"
ORBIT_NUMBER                       = 4138
PRODUCER_ID                        = LRO_LROC_TEAM
PRODUCT_CREATION_TIME              = 2013-09-16T20:08:12
PRODUCER_INSTITUTION_NAME          = "ARIZONA STATE UNIVERSITY"
PRODUCT_TYPE                       = EDR
PRODUCT_VERSION_ID                 = "v1.8"
UPLOAD_ID                          = "SC_2010140_0000_A_V01.txt"

/*DATA DESCRIPTION*/
TARGET_NAME                        = "MOON"
RATIONALE_DESC                     = "TARGET OF OPPORTUNITY"
FRAME_ID                           = RIGHT
DATA_QUALITY_ID                    = "0"
DATA_QUALITY_DESC                  = "The DATA_QUALITY_ID is set to an 8-bit
   value that encodes the following data quality information for the 
   observation. For each bit  a value of 0 means FALSE and a value of 1 means 
   TRUE. More information about the data quality ID can be found in the LROC
   EDR/CDR SIS, section 3.3 'Label and Header Descriptions'.
       Bit 1: Temperature of focal plane array is out of bounds.
       Bit 2: Threshold for saturated pixels is reached.
       Bit 3: Threshold for under-saturated pixels is reached.
       Bit 4: Observation is missing telemetry packets.
       Bit 5: SPICE information is bad or missing.
       Bit 6: Observation or housekeeping information is bad or missing.
       Bit 7: Spare.
       Bit 8: Spare."

/*ENVIRONMENT*/
LRO:TEMPERATURE_SCS                = 4.16 <degC>
LRO:TEMPERATURE_FPA                = 17.69 <degC>
LRO:TEMPERATURE_FPGA               = -10.73 <degC>
LRO:TEMPERATURE_TELESCOPE          = 7.20 <degC>
LRO:TEMPERATURE_SCS_RAW            = 2756
LRO:TEMPERATURE_FPA_RAW            = 2116
LRO:TEMPERATURE_FPGA_RAW           = 3364
LRO:TEMPERATURE_TELESCOPE_RAW      = 2615

/*IMAGING PARAMETERS*/
CROSSTRACK_SUMMING                 = 1
BANDWIDTH                          = 300 <nm>
CENTER_FILTER_WAVELENGTH           = 600 <nm>
LINE_EXPOSURE_DURATION             = 0.337600 <ms>
LRO:LINE_EXPOSURE_CODE             = 0
LRO:DAC_RESET_LEVEL                = 188
LRO:CHANNEL_A_OFFSET               = 70
LRO:CHANNEL_B_OFFSET               = 123
LRO:COMPAND_CODE                   = 3
LRO:LINE_CODE                      = 51
LRO:BTERM                          = (0,16,69,103,128)
LRO:MTERM                          = (0.5,0.25,0.125,0.0625,0.03125)
LRO:XTERM                          = (0,64,424,536,800)
LRO:COMPRESSION_FLAG               = 1
LRO:MODE                           = 7

/*DATA OBJECT*/
OBJECT                             = IMAGE
    LINES                          = 52224
    LINE_SAMPLES                   = 5064
    SAMPLE_BITS                    = 8
    SAMPLE_TYPE                    = LSB_INTEGER
    UNIT                           = "RAW_INSTRUMENT_COUNT"
    MD5_CHECKSUM                   = "22c603cf339efe8a77c16696372e33c7"
END_OBJECT                         = IMAGE
END
"""

In [4]:
url = "https://astroservices.usgs.gov/pfeffer/v1/pds/"
resp = requests.post(url, json={"label":label})

# File to write the returned ISD information to:
isd_file = '/scratch/csm2020/M128962330RE.json'

# Write the pfeffer-generated ISD to a file:
with open(isd_file, 'w') as f:
    d = resp.json()
    json.dump(d, f)

In [5]:
plugin = csmapi.Plugin.findPlugin('UsgsAstroPluginCSM')
isd = csmapi.Isd(isd_file)
model_name = resp.json()['name_model']

In [6]:
model = plugin.constructModelFromISD(isd, model_name)

In [7]:
if plugin.canModelBeConstructedFromISD(isd, model_name):
    model = plugin.constructModelFromISD(isd, model_name)

In [8]:
isize = model.getImageSize()
isize.line, isize.samp

(52224.0, 5064.0)

In [9]:
ic = csmapi.ImageCoord(100, 100)
gnd = model.imageToGround(ic, 100)
gnd.x, gnd.y, gnd.z

(414520.47646492894, 474963.55179077573, -1619101.1855537565)